In [3]:
import pandas as pd
import numpy as np
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_colwidth = 1000

In [6]:
import re

In [2]:
#!pip install google-cloud-storage
import pandas as pd
from google.cloud import storage
# import os

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Hedwig/00_Data/msca310019-capstone-49b3-6876cdba6fac.json"

ModuleNotFoundError: No module named 'google.cloud'

In [2]:
#!pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
client = storage.Client()
bucket = client.get_bucket('user-scripts-msca310019-capstone-49b3')

In [3]:
blob = bucket.blob('data/conversation_emails.csv')
content = blob.download_as_string()

In [3]:
import io
#df = pd.read_csv(io.BytesIO(content))
df = pd.read_csv('/Users/scottsmacbook/capstone/00_Data/data_conversation_emails.csv')

In [4]:
df.drop_duplicates(subset=['message'], keep='first',inplace=True)

In [5]:
thread_counts=df.groupby(['thread'], as_index=False)['message'].agg('count')
thread_counts.head(20)

,thread,message
0,0,8432
1,1,2546
2,2,68
3,3,137
4,4,1
5,5,2
6,6,2
7,7,3
8,8,3
9,9,4


In [6]:
thread_multi=thread_counts[thread_counts.message>=2]
thread_multi.drop([0,1], inplace = True)

/var/folders/p4/82nd8dg50fq_lzgh5s75s36h0000gn/T/ipykernel_56385/2027695490.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thread_multi.drop([0,1], inplace = True)


In [7]:
multi_threads_list=thread_multi.thread.to_list()

In [8]:
df['flg_multi']=df['thread'].apply(lambda x: 1 if x in multi_threads_list else 0)

In [9]:
df_message_v3=df[df.flg_multi==1]
df_message_v3.head()

,Unnamed: 0,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
2,2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,2,0,[3],[],[],test successful. way to go!!!,1
4,4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.,1
5,5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],"Greg, How about either next Tuesday or Thursda...",1
8,8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",Re: High Speed Internet Access,5,0,[9],[],[],1. login: pallen pw: ke9davis I don't think th...,1
9,9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",FW: fixed forward or other Collar floor gas pr...,6,0,[10],[],[],---------------------- Forwarded by Phillip K ...,1


In [10]:
df_message_v3['time'] = pd.to_datetime(df_message_v3['time'])
df_message_v3

/var/folders/p4/82nd8dg50fq_lzgh5s75s36h0000gn/T/ipykernel_56385/3744740549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_message_v3['time'] = pd.to_datetime(df_message_v3['time'])


,Unnamed: 0,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
2,2,2000-10-18 03:00:00-07:00,Re: test,2,0,[3],[],[],test successful. way to go!!!,1
4,4,2000-08-31 05:07:00-07:00,Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.,1
5,5,2000-08-31 04:17:00-07:00,Re: Hello,3,0,[5],[],[],"Greg, How about either next Tuesday or Thursda...",1
8,8,2000-10-17 02:26:00-07:00,Re: High Speed Internet Access,5,0,[9],[],[],1. login: pallen pw: ke9davis I don't think th...,1
9,9,2000-10-16 06:44:00-07:00,FW: fixed forward or other Collar floor gas pr...,6,0,[10],[],[],---------------------- Forwarded by Phillip K ...,1
...,...,...,...,...,...,...,...,...,...,...
245121,517393,2001-11-30 14:53:21-08:00,RE:,208,322,[51654],[],[],"not yet, rob has been in the court house all d...",1
245122,517394,2001-11-29 11:41:51-08:00,FW: Hello,11292,322,[51654],[],[],"-----Original Message----- From: \t""Lerner, L...",1
245123,517395,2001-11-29 11:04:24-08:00,RE:,208,322,[51636],[],[],"very busy, jess is ok just scared -----Origina...",1
245126,517398,2001-11-28 07:20:00-08:00,RE: CONFIDENTIAL,12455,322,[3541],[],[],"2 -----Original Message----- From: \tDoucet, D...",1


In [11]:
df_message_v3.sort_values(by=['thread','time'], ascending=[True,True], inplace=True)

/var/folders/p4/82nd8dg50fq_lzgh5s75s36h0000gn/T/ipykernel_56385/2072996508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_message_v3.sort_values(by=['thread','time'], ascending=[True,True], inplace=True)


In [12]:
import ast
df_message_v3['recipient'] = df_message_v3['recipient'].apply(lambda x: ast.literal_eval(x))

/var/folders/p4/82nd8dg50fq_lzgh5s75s36h0000gn/T/ipykernel_56385/3840432164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_message_v3['recipient'] = df_message_v3['recipient'].apply(lambda x: ast.literal_eval(x))


In [13]:
test = df_message_v3[df_message_v3.thread<31]

In [62]:
#test = test.explode('recipient')

In [14]:
df_message_v3.thread.unique()

array([     2,      3,      5, ..., 137844, 137870, 137877])

In [66]:
threads = df_message_v3.thread.unique()

message_replies = []
for thread in threads:
    thread_df = df_message_v3[df_message_v3.thread==thread]
    thread_df.drop(columns = ['Unnamed: 0','cc','bcc','flg_multi'],inplace=True)
    
    r = list(thread_df.recipient)
    s = list(thread_df.sender)
    
    pairs = []

    for i in range(len(thread_df)):
        for j in range(i,-1,-1):
            if(s[i] in r[j]):
                if(s[j] in r[i]):
                    pairs.append((j,i))
                    break

    for pair in pairs:
        message = thread_df.iloc[pair[0]]
        reply = thread_df.iloc[pair[1]]
        reply_new = reply.add_prefix('reply_')
        concatenated_row = pd.concat([message, reply_new], axis=0)
        message_replies.append(concatenated_row)
#     print(pairs)

In [58]:
message_replies_df = pd.DataFrame(data=message_replies)

In [62]:
message_replies_df.drop(columns = ['reply_subject','reply_thread'],inplace=True)

In [60]:
message_replies_df[message_replies_df.thread!=2]

,time,subject,thread,sender,recipient,message,reply_time,reply_subject,reply_thread,reply_sender,reply_recipient,reply_message
Unnamed 17,2000-10-06 00:42:00-07:00,RE: Hello,3,13140,[18237],the problem is that I need to go at noon.,2000-10-06 00:45:00-07:00,RE: Hello,3,18237,[13140],I'm more then happy to take you if you still n...
Unnamed 18,2000-10-06 00:42:00-07:00,RE: Hello,3,13140,[18237],the problem is that I need to go at noon.,2000-10-06 00:51:00-07:00,RE: Hello,3,18237,[13140],call me at work... 223-5600 x7804 -----Origina...
Unnamed 19,2001-05-01 09:42:00-07:00,RE: Hello,3,617,[4140],"Molly, Kim Watson would like us to bring Jacob...",2001-05-01 09:51:00-07:00,RE: Hello,3,4140,[617],Sounds great--I'll coordinate with Shirley. Ja...
Unnamed 20,2001-05-01 09:51:00-07:00,RE: Hello,3,4140,[617],Sounds great--I'll coordinate with Shirley. Ja...,2001-05-01 19:42:00-07:00,RE: Hello,3,617,[4140],"Molly, Kim Watson would like us to bring Jacob..."
Unnamed 21,2001-05-01 19:42:00-07:00,RE: Hello,3,617,[4140],"Molly, Kim Watson would like us to bring Jacob...",2001-05-01 19:51:00-07:00,RE: Hello,3,4140,[617],Sounds great--I'll coordinate with Shirley. Ja...
...,...,...,...,...,...,...,...,...,...,...,...,...
Unnamed 24661,2002-01-31 09:26:01-08:00,Canadian Trader List,137631,56,[322],"John, Please review the list below and let me ...",2002-01-31 09:34:39-08:00,RE: Canadian Trader List,137631,322,[56],Stephane Brodeur should be removed from the li...
Unnamed 24662,2002-01-31 09:34:39-08:00,RE: Canadian Trader List,137631,322,[56],Stephane Brodeur should be removed from the li...,2002-01-31 10:35:06-08:00,RE: Canadian Trader List,137631,56,[322],Thank you. Please let me know about Chris Lamb...
Unnamed 24663,2002-01-31 06:44:41-08:00,RE: Physical Gas,137634,322,[551],that was our plan but I am not sure whether UB...,2002-01-31 06:49:52-08:00,RE: Physical Gas,137634,551,[322],"John, I understand Peter Keohane is here today..."
Unnamed 24664,2002-01-30 09:40:23-08:00,Estate DPR Reporting for Power,137640,3595,"[3421, 322]",Peggy & John The estate's DPR reporting requir...,2002-01-30 10:09:47-08:00,RE: Estate DPR Reporting for Power,137640,322,[3595],"ok -----Original Message----- From: Reeves, Ka..."


In [64]:
message_replies_df.to_csv('/Users/scottsmacbook/capstone/00_Data/message_reply_pairs.csv',index=False)

In [65]:
message_replies_df[message_replies_df.thread==69]

,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
Unnamed 73,2000-10-25 03:42:00-07:00,Re: lunch,69,1340,[38500],"Hey, This week is probably not a good week, bu...",2000-10-25 04:51:00-07:00,38500,[1340],Great! I am best at scheduling things last min...
Unnamed 74,2000-11-27 03:04:00-08:00,RE: lunch,69,42586,[1691],Thurs.? -----Original Message----- From: Sara....,2000-11-27 05:15:00-08:00,1691,[42586],Thursday sounds great! I'll call you in the mo...
191581,2000-12-04 02:12:00-08:00,RE: lunch,69,18483,"[162, 18485, 18483]",Sorry to hear you're suffering. I'm flexible o...,2000-12-04 02:12:00-08:00,18483,"[162, 18485, 18483]",Sorry to hear you're suffering. I'm flexible o...
Unnamed 75,2001-10-05 09:01:41-07:00,RE: lunch,69,8558,[8600],Yes sure. I won't be here on Monday and possib...,2001-10-05 09:02:03-07:00,8600,[8558],sounds great. :) -----Original Message----- Fr...
Unnamed 76,2001-10-19 09:15:25-07:00,RE: lunch,69,1405,[24797],Shawn What about Tuesday next week? Vince ----...,2001-10-19 09:20:12-07:00,24797,[1405],"Vince, that sounds great. Should I drive by an..."
Unnamed 77,2001-10-19 09:20:12-07:00,RE: lunch,69,24797,[1405],"Vince, that sounds great. Should I drive by an...",2001-10-19 09:36:28-07:00,1405,[24797],"Shawn, Thanks. Tuesday next week, 12:00, it is..."
Unnamed 78,2001-11-19 14:36:01-08:00,RE: lunch,69,253,[43306],Sounds good. 11:30? -----Original Message-----...,2001-11-19 14:36:24-08:00,43306,[253],11:30 it is! -----Original Message----- From: ...
Unnamed 79,2001-11-20 06:24:30-08:00,RE: lunch,69,529,[20627],"I am going to Las Vegas for Thanksgiving, but ...",2001-11-20 06:40:13-08:00,20627,[529],I am open that week. How about Tuesday? From:\...
Unnamed 80,2001-11-20 06:40:13-08:00,RE: lunch,69,20627,[529],I am open that week. How about Tuesday? From:\...,2001-11-20 06:53:20-08:00,529,[20627],Good. Lets talk that morning. -----Original Me...
Unnamed 81,2001-10-19 09:20:12-07:00,RE: lunch,69,24797,[1405],"Vince, that sounds great. Should I drive by an...",2001-12-05 08:35:20-08:00,1405,[24797],"Shawn, I was about to call you. I have an emer..."


In [53]:
message_replies_df.thread.unique()[7]

69

-----------------
## Cleaning message-reply pairs

In [83]:
df_message_v4=pd.read_csv('/Users/kshitijmittal/Documents/UChicago Acad/Hedwig_AI/Hedwig/00_Data/data_message_reply_pairs.csv')

In [84]:
df_message_v4.head()

,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],"How about this Friday ? Julie has not left yet. She was 2 days away from leaving to start school on June 7th when she, quite by accident, looked at the school schedule for the first time in several weeks. Now mind you she had lined up a babysitter ,lined up a place to stay, arranged to get a key----to start the 7th. The schedule says the school starts the 14th. Pretty funny. How many years will it take her to live this down?. \tEnron Capital & Trade Resources Corp. \t \tFrom: ""Radack, Judge Sherry (DCA)"" <Radack@DCA.CO.HARRIS.TX.US> 06/09/99 10:06 AM \t To: Richard B Sanders/HOU/ECT cc: Subject: RE: test when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-06-09 08:06:00-07:00,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch."
1,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-06-10 03:54:00-07:00,5552,[40034],Today is bad. Tommorrow I will call you.
2,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-11-23 01:38:00-08:00,5552,[40034],Do you have lunch plans today?
3,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-11-23 03:13:00-08:00,5552,[40034],Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.
4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.


In [85]:
# Removing newlines and tabs
df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:\n)','',str(x)))
df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:\t)','',str(x)))
df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:\>)','',str(x)))


df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:\n)','',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:\t)','',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:\>)','',str(x)))

In [86]:
df_message_v4.head()

,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],"How about this Friday ? Julie has not left yet. She was 2 days away from leaving to start school on June 7th when she, quite by accident, looked at the school schedule for the first time in several weeks. Now mind you she had lined up a babysitter ,lined up a place to stay, arranged to get a key----to start the 7th. The schedule says the school starts the 14th. Pretty funny. How many years will it take her to live this down?. Enron Capital & Trade Resources Corp. From: ""Radack, Judge Sherry (DCA)"" <Radack@DCA.CO.HARRIS.TX.US 06/09/99 10:06 AM To: Richard B Sanders/HOU/ECT cc: Subject: RE: test when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-06-09 08:06:00-07:00,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch."
1,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-06-10 03:54:00-07:00,5552,[40034],Today is bad. Tommorrow I will call you.
2,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-11-23 01:38:00-08:00,5552,[40034],Do you have lunch plans today?
3,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-11-23 03:13:00-08:00,5552,[40034],Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.
4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],"when? how are you and your family? is julie gone? -----Original Message----- From: Richard B Sanders [mailto:rsander@ect.enron.com] Sent: Tuesday, June 08, 1999 3:52 PM To: Radack, Judge Sherry Subject: Re: test Lets get together for lunch.",1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.


In [87]:
# Removing content after original message
df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(-----Original\sMessage-----)([\w\W\d\D\s\S]*)','',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(-----Original\sMessage-----)([\w\W\d\D\s\S]*)','',str(x)))
# df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:\n)','',str(x)))
# df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:\t)','',str(x)))

df_message_v4.head()

,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],"How about this Friday ? Julie has not left yet. She was 2 days away from leaving to start school on June 7th when she, quite by accident, looked at the school schedule for the first time in several weeks. Now mind you she had lined up a babysitter ,lined up a place to stay, arranged to get a key----to start the 7th. The schedule says the school starts the 14th. Pretty funny. How many years will it take her to live this down?. Enron Capital & Trade Resources Corp. From: ""Radack, Judge Sherry (DCA)"" <Radack@DCA.CO.HARRIS.TX.US 06/09/99 10:06 AM To: Richard B Sanders/HOU/ECT cc: Subject: RE: test when? how are you and your family? is julie gone?",1999-06-09 08:06:00-07:00,40034,[5552],when? how are you and your family? is julie gone?
1,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-06-10 03:54:00-07:00,5552,[40034],Today is bad. Tommorrow I will call you.
2,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 01:38:00-08:00,5552,[40034],Do you have lunch plans today?
3,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 03:13:00-08:00,5552,[40034],Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.
4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.


In [88]:
# Removing content after From: message
df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(From\:)([\w\W\d\D\s\S]*)','',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(From\:)([\w\W\d\D\s\S]*)','',str(x)))

# ----- Forwarded by
# df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:\n)','',str(x)))
# df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:\t)','',str(x)))
df_message_v4.head()

,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
0,1999-06-09 04:18:00-07:00,RE: test,2,5552,[40034],"How about this Friday ? Julie has not left yet. She was 2 days away from leaving to start school on June 7th when she, quite by accident, looked at the school schedule for the first time in several weeks. Now mind you she had lined up a babysitter ,lined up a place to stay, arranged to get a key----to start the 7th. The schedule says the school starts the 14th. Pretty funny. How many years will it take her to live this down?. Enron Capital & Trade Resources Corp.",1999-06-09 08:06:00-07:00,40034,[5552],when? how are you and your family? is julie gone?
1,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-06-10 03:54:00-07:00,5552,[40034],Today is bad. Tommorrow I will call you.
2,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 01:38:00-08:00,5552,[40034],Do you have lunch plans today?
3,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 03:13:00-08:00,5552,[40034],Really? I'd feel like a mooch. Lets have lunch next week. Any day but Monday or Thurs.
4,1999-06-09 08:06:00-07:00,RE: test,2,40034,[5552],when? how are you and your family? is julie gone?,1999-11-23 03:58:00-08:00,5552,[40034],Tues.is good. I'll call you.


In [89]:
# Removing content after Forwarded by message
df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(-----\sForwarded\sby)([\w\W\d\D\s\S]*)','',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(-----\sForwarded\sby)([\w\W\d\D\s\S]*)','',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(cc:)([\w\W\d\D\s\S]*)','',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(cc:)([\w\W\d\D\s\S]*)','',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(Subject:)([\w\W\d\D\s\S]*)','',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(Subject:)([\w\W\d\D\s\S]*)','',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(To:)([\w\W\d\D\s\S]*)','',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(To:)([\w\W\d\D\s\S]*)','',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:[\*]+)',' ',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:[\*]+)',' ',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:[\-]+)',' ',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:[\-]+)',' ',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:[\?]{2,})',' ',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:[\?]{2,})',' ',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:[\.]{2,})',' ',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:[\.]{2,})',' ',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:[\_]+)',' ',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:[\_]+)',' ',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:[\!]{2,})',' ',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:[\!]{2,})',' ',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:[\:]{2,})',' ',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:[\:]{2,})',' ',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:[\;]{2,})',' ',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:[\;]{2,})',' ',str(x)))

df_message_v4['message']=df_message_v4['message'].apply(lambda x: re.sub(r'(?:[\,]{2,})',' ',str(x)))
df_message_v4['reply_message']=df_message_v4['reply_message'].apply(lambda x: re.sub(r'(?:[\,]{2,})',' ',str(x)))

In [90]:
pd.DataFrame(df_message_v4.groupby(['thread','subject'], as_index=False)['time'].agg('count').sort_values(by='time',ascending=False)).head(40)

,thread,subject,time
21,208,RE:,2585
10555,89437,Schedule Crawler: HourAhead Failure,343
1881,17201,Schedule Crawler: HourAhead Failure <CODESITE>,229
1796,16298,Synchronization Log:,153
13961,123277,Re: apb checkout,76
13981,123296,Re: apb,47
89,669,FW:,44
11528,96913,Schedule Crawler: DayAhead Failure,40
2,3,RE: Hello,37
13962,123277,apb checkout,31


In [91]:
df_message_v4[df_message_v4.thread==208]

,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
128,2000-02-04 19:12:00-08:00,RE:,208,15082,[2027],"Daren, Just call me Mike. How about Thursday, the 17th at 1:00. Let's plan on 1 hour. Can you locate a conference room? Mike",2000-02-06 23:18:00-08:00,2027,[15082],"Mike, Thursday the 17th at 1:00 wil work for me. I have reserved room 3125b. We can meet at my desk (3138C) and walk over there. Daren"
129,2000-06-05 10:12:00-07:00,RE:,208,38495,[1340],"I have a meeting with Dave on Monday at 8:30am. We will be bringing the deal guys (Bill Grealis, Chuck Winger, Art Vivar, myself and possibly one other) so we're ready to listen and make something shake loose. I am also going to contact Rich Kinder this week to talk about the possibilities. I very much appreciate your input and your insight and would welcome your help when the time is right. As a side note you may have heard that Madeleine Ludlow has left to pursue other business interests and we are in the hunt for a corporate CFO and talent as I mentioned is my number one priority. As to your inclusion in this meeting I will inquire with Dave as to the attendees and mention your name as a good participant. (Dave's call obviously). Thanks again",2000-06-06 07:01:00-07:00,1340,[38495],"Mike: Just a thought regarding Kinder/Morgan. These guys mainly care about the gas pipelines and that some of there projects are actually put on there lines to increase throughput. These plants that they want on their lines are currently in construction or development. Their only operational plants are in Colorado. They have a good generation team, but I fell that they do not want to be in the business of operating and maintaining the plants, just as long as somebody wants to put them along their lines. If you guys offered to buy their plants and projects, they would mainly care about price (cash and stock in the new company) and to put contracts in place for some of their projects to put on their lines. Hope this helps. Ben"
130,2000-06-06 07:01:00-07:00,RE:,208,1340,[38495],"Mike: Just a thought regarding Kinder/Morgan. These guys mainly care about the gas pipelines and that some of there projects are actually put on there lines to increase throughput. These plants that they want on their lines are currently in construction or development. Their only operational plants are in Colorado. They have a good generation team, but I fell that they do not want to be in the business of operating and maintaining the plants, just as long as somebody wants to put them along their lines. If you guys offered to buy their plants and projects, they would mainly care about price (cash and stock in the new company) and to put contracts in place for some of their projects to put on their lines. Hope this helps. Ben",2000-08-10 08:25:00-07:00,38495,[1340],I did and I am in the process of reviewing possibilities. Will look for your additional information.
131,2000-04-17 06:28:00-07:00,RE:,208,10047,[151],"<<One way NDA.doc Cameron Sellers Vice President, Business Development PERFECT.COM 1860 Embarcadero Road Suite 210 Palo Alto, CA 94303 cameron@perfect.com 650.858.7719 (direct dial) 650.858.1095 (fax) 650.208.9455 (cell)",2000-08-31 08:02:00-07:00,151,[10047],"Hopefully you mean ""buy?"" Cameron Sellers <cameron@perfect.com on 08/31/2000 02:50:39 PM"
132,2000-04-17 06:28:00-07:00,RE:,208,10047,[151],"<<One way NDA.doc Cameron Sellers Vice President, Business Development PERFECT.COM 1860 Embarcadero Road Suite 210 Palo Alto, CA 94303 cameron@perfect.com 650.858.7719 (direct dial) 650.858.1095 (fax) 650.208.9455 (cell)",2000-08-31 08:05:00-07:00,151,[10047],Sell. Cameron Sellers <cameron@perfect.com on 08/31/2000 03:07:36 PM
...,...,...,...,...,...,...,...,...,...,...
2710,2002-05-22 08:23:55-07:00,RE:,208,16321,[13150],"Lisa Still fighting with the Mo Jo huh How is everything going, are you able to leave with a smile on your face everyday? Awesome that Susie is coming 

In [92]:
df_message_v4[df_message_v4.thread==89437]

,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
19399,2001-12-11 16:57:09-08:00,Schedule Crawler: HourAhead Failure,89437,8567,[8567],Start Date: 12/11/01; HourAhead hour: 15; HourAhead schedule download failed. Manual intervention required.,2001-12-11 16:57:09-08:00,8567,[8567],Start Date: 12/11/01; HourAhead hour: 15; HourAhead schedule download failed. Manual intervention required.
19400,2001-12-11 19:57:13-08:00,Schedule Crawler: HourAhead Failure,89437,8567,[8567],Start Date: 12/11/01; HourAhead hour: 18; HourAhead schedule download failed. Manual intervention required.,2001-12-11 19:57:13-08:00,8567,[8567],Start Date: 12/11/01; HourAhead hour: 18; HourAhead schedule download failed. Manual intervention required.
19401,2001-12-11 22:56:55-08:00,Schedule Crawler: HourAhead Failure,89437,8567,[8567],Start Date: 12/11/01; HourAhead hour: 21; HourAhead schedule download failed. Manual intervention required.,2001-12-11 22:56:55-08:00,8567,[8567],Start Date: 12/11/01; HourAhead hour: 21; HourAhead schedule download failed. Manual intervention required.
19402,2001-12-12 08:56:53-08:00,Schedule Crawler: HourAhead Failure,89437,8567,[8567],Start Date: 12/12/01; HourAhead hour: 7; HourAhead schedule download failed. Manual intervention required.,2001-12-12 08:56:53-08:00,8567,[8567],Start Date: 12/12/01; HourAhead hour: 7; HourAhead schedule download failed. Manual intervention required.
19403,2001-12-12 12:57:11-08:00,Schedule Crawler: HourAhead Failure,89437,8567,[8567],Start Date: 12/12/01; HourAhead hour: 11; HourAhead schedule download failed. Manual intervention required.,2001-12-12 12:57:11-08:00,8567,[8567],Start Date: 12/12/01; HourAhead hour: 11; HourAhead schedule download failed. Manual intervention required.
...,...,...,...,...,...,...,...,...,...,...
19737,2002-02-06 08:56:38-08:00,Schedule Crawler: HourAhead Failure,89437,8567,[8567],Start Date: 2/6/02; HourAhead hour: 11; HourAhead schedule download failed. Manual intervention required.,2002-02-06 08:56:38-08:00,8567,[8567],Start Date: 2/6/02; HourAhead hour: 11; HourAhead schedule download failed. Manual intervention required.
19738,2002-02-06 09:56:04-08:00,Schedule Crawler: HourAhead Failure,89437,8567,[8567],Start Date: 2/6/02; HourAhead hour: 12; HourAhead schedule download failed. Manual intervention required.,2002-02-06 09:56:04-08:00,8567,[8567],Start Date: 2/6/02; HourAhead hour: 12; HourAhead schedule download failed. Manual intervention required.
19739,2002-02-06 10:56:18-08:00,Schedule Crawler: HourAhead Failure,89437,8567,[8567],Start Date: 2/6/02; HourAhead hour: 13; HourAhead schedule download failed. Manual intervention required.,2002-02-06 10:56:18-08:00,8567,[8567],Start Date: 2/6/02; HourAhead hour: 13; HourAhead schedule download failed. Manual intervention required.
19740,2002-02-06 11:56:31-08:00,Schedule Crawler: HourAhead Failure,89437,8567,[8567],Start Date: 2/6/02; HourAhead hour: 14; HourAhead schedule download failed. Manual intervention required.,2002-02-06 11:56:31-08:00,8567,[8567],Start Date: 2/6/02; HourAhead hour: 14; HourAhead schedule download failed. Manual intervention required.


In [93]:
df_message_v4.shape

(27463, 10)

In [94]:
df_message_v5 = df_message_v4[df_message_v4.message != df_message_v4.reply_message]
df_message_v5.shape

(24640, 10)

In [95]:
pd.DataFrame(df_message_v5.groupby(['thread','subject'], as_index=False)['time'].agg('count').sort_values(by='time',ascending=False)).head(40)

,thread,subject,time
20,208,RE:,2570
12930,123277,Re: apb checkout,76
12950,123296,Re: apb,47
2,3,RE: Hello,37
12931,123277,apb checkout,31
12990,123342,Re: natsource checkout,29
60,519,RE: Hi,27
664,5094,Congratulations,25
12951,123296,apb,25
2395,22340,Re: Teams,24


In [98]:
df_message_v5.sample(5)

,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
2746,2000-12-13 08:22:00-08:00,Final Filed Version -- SDG&E Comments,317,203,"[211, 261, 86]",FYI.,2000-12-14 03:27:00-08:00,86,[203],Whoever wrote this did an excellent job
6842,2001-07-11 13:51:36-07:00,RE: PRM SLICE MEeting,19162,1521,[1687],"For you Stewart, anything.",2001-07-11 13:53:10-07:00,1687,[1521],wow I did this for you to prevent another airplane experience
4283,2001-12-11 09:22:01-08:00,Free agents,4381,2263,[371],"Eric: I trust everything is going well as Enron. I am actively pursuing a job out here at Mcdonalds. ""Would you like fries with that"" I practice on a regular basis. In any case, I would like to pick up the Patriots Defense and the Steelers Defense as free agent pick ups, if still possible. I would like to drop Engram and the Titans defense. Please let me know. Thanks Harry Bucalo 713 334 3543",2001-12-13 09:23:36-08:00,371,[2263],"Harry, Sorry it has taken me so long to get back to you. Things are so bad at work right now. I have nothing to do whasoever. I get in to work around 9 am and leave around 12:30 every day. I hope all is going well in the job search. I saw your add/drop request for the Pats. If you want me to add the Steelers as well just let me know. Also, you might want to practice, ""Would you like to SuperSize that?"" Eric"
13405,2001-04-11 04:10:00-07:00,Re: SWAP,55705,1910,[1930],Is this to be in replacement of the amendment I previously drafted or in addition to the volumes on that amendment? Janet H Wallis/ENRON@enronXgate 04/11/2001 11:07 AM,2001-04-11 04:13:00-07:00,1930,[1910],"Good Question, it is in addition too the previous volumes."
14733,2000-10-11 07:12:00-07:00,CP3107,59518,14364,[1721],Here is a copy of the confirmation letter prepared for Encore. Let me know if there is anything else I can do. Dianne,2000-10-11 08:13:00-07:00,1721,[14364],"Thanks, I appreciate your help! Dianne Seib 10/11/2000 03:12 PM"


In [99]:
df_message_v5.to_csv('/Users/kshitijmittal/Documents/UChicago Acad/Hedwig_AI/Hedwig/00_Data/data_message_reply_pairs_cleaned.csv')

In [101]:
len(df_message_v5['sender'].unique())

3576